In [1]:
from re import VERBOSE
# This line is very important on Colab. 
# Without it, Tensorflow 2.x will run and PINNs will not work.
%tensorflow_version 1.x

%xmode VERBOSE
%pdb on

TensorFlow 1.x selected.
Exception reporting mode: Verbose
Automatic pdb calling has been turned ON


In [2]:
import sys

!pip install ipdb
import ipdb
sys.path.insert(0, '../../Utilities/')

import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
# from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time

# Animation imports
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib.pyplot as plt

print(tf.test.gpu_device_name())


using_gpu = tf.test.is_gpu_available(
  cuda_only=False, min_cuda_compute_capability=None
)

print(using_gpu)

if using_gpu:
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)


/device:GPU:0
True
Sun Mar  6 01:22:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    35W / 250W |    257MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                     

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct  9 20:11:57 2017

@author: mraissi
"""

import numpy as np
import matplotlib as mpl
#mpl.use('pgf')

def figsize(scale, nplots = 1):
    fig_width_pt = 390.0                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = nplots*fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(1.0),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }
mpl.rcParams.update(pgf_with_latex)

import matplotlib.pyplot as plt

# I make my own newfig and savefig functions
def newfig(width, nplots = 1):
    fig = plt.figure(figsize=figsize(width, nplots))
    ax = fig.add_subplot(111)
    return fig, ax

def savefig(filename, crop = True):
    if crop == True:
#        plt.savefig('{}.pgf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.pdf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.eps'.format(filename), bbox_inches='tight', pad_inches=0)
    else:
#        plt.savefig('{}.pgf'.format(filename))
        plt.savefig('{}.pdf'.format(filename))
        plt.savefig('{}.eps'.format(filename))

In [5]:
np.random.seed(1234)
#tf.set_random_seed(1234)

class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.t = X[:,2:3]
        self.u = u
        
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
       #self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     #log_device_placement=True))
        
        # Initialize parameters
        self.alpha_1 = tf.Variable([-1.8], dtype=tf.float32)
        #self.lambda_2 = tf.Variable([-6.0], dtype=tf.float32)
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
                
        self.u_pred = self.net_u(self.x_tf,self.y_tf, self.t_tf)
        self.f_pred = self.net_f(self.x_tf,self.y_tf, self.t_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        self.optimizer_Adam = tf.train.AdamOptimizer(learning_rate=0.01) # try both a larger and smaller rate
        # self.optimizer_Adam = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases, eps=1e-3):
        num_layers = len(weights) + 1

        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
          W = weights[l]
          b = biases[l]
          H = tf.nn.relu(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        
        
            # W = tf.Variable(weights[l])
            # b = tf.Variable(biases[l])

            # z = tf.matmul(H, W)
            # batch_mean, batch_var = tf.nn.moments(z,[0])
            # ipdb.set_trace()
            # scale = tf.Variable(tf.ones([100]))
            # beta = tf.Variable(tf.zeros([100]))
            # z_hat = tf.nn.batch_normalization(z,batch_mean,batch_var,beta,scale,eps)
            # z = (z - batch_mean) / tf.sqrt(batch_var + eps) # eps = 1e^-3
            # H = tf.tanh(tf.add(tf.matmul(z, W), b))
            # H = tf.nn.relu(tf.add(tf.matmul(z_hat, W), b))
        #     H = tf.nn.relu(tf.add(z, b))
        # W = weights[-1]
        # b = biases[-1]
        # Y = tf.add(tf.matmul(H, W), b)
        return Y
            
    def net_u(self,x,y,t):  
        u = self.neural_net(tf.concat([x,y,t],1), self.weights, self.biases)
        return u
    
    def net_f(self, x,y,t):
        alpha_1 = self.alpha_1        
        u = self.net_u(x,y,t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        f = u_t - alpha_1*(u_xx + u_yy)
        return f
    
    def callback(self, self.loss, self.alpha_1):
        print('Loss: %e, l1: %.5f' % (self.loss,  self.alpha_1))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x,self.y_tf: self.y, self.t_tf: self.t, self.u_tf: self.u}
        
        pinn_df = open('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab/PINN_output.txt', 'w')

        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print            
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                alpha_1_value = self.sess.run(self.alpha_1)
                #lambda_2_value = np.exp(self.sess.run(self.lambda_2))
                
                print('It: %d, Loss: %.3e, alpha_1: %.3f, Time: %.2f' % 
                      (it, loss_value, alpha_1_value, elapsed))
                
                pinn_df.write('It: %d, Loss: %.3e, alpha_1: %.3f, Time: %.2f \n' % 
                      (it, loss_value, alpha_1_value, elapsed))
                
                start_time = time.time()
        
        pinn_df.close()
        
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss, self.alpha_1],
                                loss_callback = self.callback)
        
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star[:,0:1],self.y_tf: X_star[:,1:2], self.t_tf: X_star[:,2:3]}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [6]:
#if __name__ == "__main__": 
     
    #nu = 0.01/np.pi

N_u = 1000000# 853000
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 1]
    
    # data = scipy.io.loadmat('../Data/heat_eq.mat')
    
    #t = data['t'].flatten()[:,None]
    #x = data['x'].flatten()[:,None]
   # Exact = np.real(data['usol']).T
    
    #X, T = np.meshgrid(x,t)
    
input = np.load('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab/input.npy') # X1, X2 and T as three columns
output = np.load('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab/output.npy') # This is the solution of 2-D heat equation
output = output.reshape(output.shape[0],1) # changes shape from (n, ) to (n, 1)
    
X_star = input #input is coming from the 2Dheat.py file
u_star = output.flatten()[:,None] #output is coming from the 2Dheat.py file 

# Domain bounds
lb = input.min(0)
ub = input.max(0)

print(lb)
print(ub)

[0. 0. 0.]
[1.         1.         0.12492188]


In [7]:
start_point = 1600
stop_point = 1844799
sa = np.linspace(start_point, stop_point, stop_point-start_point)
# t=0 values are preseved in the original dataset as they are intial conditions
idx_sample = np.random.choice(sa, N_u, replace=False)

# New train Data for 2-D Heat Equation
X_u_train = np.vstack((input[0:1600, :], input[idx_sample.astype(int), :]))
u_train = np.concatenate([output[0:1600], output[idx_sample.astype(int)]])

print(X_u_train.min(0))
print(X_u_train.max(0))

[0. 0. 0.]
[1.   1.   0.09]


In [ ]:
######################################################################
######################## Noiseless Data ##############################
######################################################################
noise = 0.0            
         
# idx = np.random.choice(input.shape[0], N_u, replace=False)
# X_u_train = input[idx,:]
# u_train = u_star[idx,:]

# np.where(input[:,2] >= 0.09)

start_point = 1600
stop_point = 1844799
sa = np.linspace(start_point, stop_point, stop_point-start_point)
# t=0 values are preseved in the original dataset as they are intial conditions
idx_sample = np.random.choice(sa, N_u, replace=False)

# New train Data for 2-D Heat Equation
X_u_train = np.vstack((input[0:1600, :], input[idx_sample.astype(int), :]))
u_train = np.concatenate([output[0:1600], output[idx_sample.astype(int)]])

# X_u_train = input[:1844800, :]
# u_train = output[:1844800]

model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
model.train(10000)#You can run for small iterations first, to see whether it is running properly

u_pred, f_pred = model.predict(X_star)
        
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)

#\U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')#This graphing part yet to be figure out.
    
alpha_1_value = model.sess.run(model.alpha_1)
#lambda_2_value = model.sess.run(model.lambda_2)
#lambda_2_value = np.exp(lambda_2_value)

error_alpha_1 = np.abs(alpha_1_value - 2)*100
#error_lambda_2 = np.abs(lambda_2_value - nu)/nu * 100


print('Error u: %e' % (error_u))    
print('Error l1: %.5f%%' % (error_alpha_1)) 

df = open('output.txt', 'w')
df.write('Error u: %e' % (error_u))  
df.write('\n')
df.close()

np.save('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab//u_train-18Feb2022-relu-2.npy', u_train)

u_pred_3D = np.reshape(u_pred, (1600, 40, 40))
np.save('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab/u_pred_3D-18Feb2022-relu-2.npy', u_pred_3D)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

It: 0, Loss: 1.175e+03, alpha_1: -1.800, Time: 1.68
It: 10, Loss: 1.108e+03, alpha_1: -1.800, Time: 3.81
It: 20, Loss: 7.131e+02, alpha_1: -1.800, Time: 3.81
It: 30, Loss: 4.959e+02, alpha_1: -1.800, Time: 3.80
It: 40, Loss: 2.737e+02, alpha_1: -1.800, Time: 3.80
It: 50, Loss: 1.477e+02, alpha_1: -1.800, Time: 3.81
It: 60, Loss: 1.249e+02, alpha_1: -1.800, Time: 3.80
It: 70, Loss: 1.117e+02, alpha_1: -1.800, Time: 3.80
It: 80, Loss: 1.034e+02, alpha_1: -1.800, Time: 3.81
It: 90, Loss: 9.790e+01, alpha_1: -1.800, Time: 3.80
It: 100, Loss: 9.175e+01, alpha_1: -1.800, Time: 3.81
It: 110, Loss: 8.541e+01, alpha_1: -1.800, Time: 3.82

In [ ]:
%debug

In [ ]:
print(u_pred)

In [ ]:
np.reshape(u_pred, (1600,40,40))[0, :, :]

NameError: ignored

In [ ]:
print(X_u_train)

In [ ]:
print(u_train)
np.save('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab//u_train-12Feb2022.npy', u_train)

In [ ]:
u_pred_3D = np.reshape(u_pred, (1600, 40, 40))
np.save('/content/drive/MyDrive/Colab Notebooks/UTRGV-Colab/u_pred_3D-12Feb2022.npy', u_pred_3D)

In [ ]:
print(u_pred_3D)

In [ ]:
######################################################################
############################ Animation ###############################
###################################################################### 
import matplotlib
from matplotlib import rc
import matplotlib.pyplot as plt
%matplotlib inline

rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']
!apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng

u_pred_3D = np.reshape(u_pred, (1600, 40, 40))

plate_length = 1  # 10
delta_x = 0.025

alpha = 2

num_x_inter = int((plate_length)/delta_x)
max_iter_time = num_x_inter * num_x_inter

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)

tick_intervals = np.linspace(0, 40, 11, retstep=True)[0]
tick_labels = np.array(
['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0'])


def plotheatmap(u_k, k):
  # Clear the current plot figure
  plt.clf()

  plt.title(f"Temperature at t = {k*delta_t:.3f} unit time")

  plt.xlabel("x")
  plt.xticks(tick_intervals, tick_labels)

  plt.ylabel("y")
  plt.yticks(tick_intervals, tick_labels)

        # This is to plot u_k (u at time-step k)
  plt.pcolormesh(u_k, cmap=plt.cm.jet, vmin=0, vmax=100)
  plt.colorbar()

  return plt


def animate(k):
  plotheatmap(u_pred_3D[k], k)


anim = animation.FuncAnimation(
    plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False)
anim.save("predicted_heat_equation_solution-07Feb2022.gif")